In [1]:
import numpy as np

from strat import RSIRisingFalling

from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.simulate import run_df_backtest, or_backtest


np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [42]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="15m",
    candles_to_dl=3000,
)

In [43]:
short_strat = RSIRisingFalling(
    long_short="short",
    rsi_length=np.array([14]),
    rsi_is_above=np.arange(50, 76, 5),
)
short_strat.set_entries_exits_array(
    candles=candles,
    ind_set_index=0,
)
short_strat.plot_signals(candles=candles)

In [4]:
long_strat = RSIRisingFalling(
    long_short="long",
    rsi_length=np.array([14]),
    rsi_is_below=np.arange(35, 61, 5),
)
long_strat.set_entries_exits_array(
    candles=candles,
    ind_set_index=2,
)
long_strat.plot_signals(candles=candles)

In [54]:
backtest_settings = BacktestSettings()

exchange_settings = ExchangeSettings(
    asset_tick_step=3,
    leverage_mode=1,
    leverage_tick_step=2,
    limit_fee_pct=0.0003,
    market_fee_pct=0.0006,
    max_asset_size=100.0,
    max_leverage=150.0,
    min_asset_size=0.001,
    min_leverage=1.0,
    mmr_pct=0.004,
    position_mode=3,
    price_tick_step=1,
)

static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

# test
# static_os = StaticOrderSettings(
#     increase_position_type=IncreasePositionType.PctAccountEntrySize,  # Adjust as needed
#     leverage_strategy_type=LeverageStrategyType.Dynamic,
#     pg_min_max_sl_bcb="min",  # Adjust as needed based on your strategy
#     sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
#     sl_to_be_bool=False,
#     starting_bar=50,
#     starting_equity=588.0,
#     static_leverage=None,  # Adjust as needed
#     tp_fee_type="market",
#     tp_strategy_type=TakeProfitStrategyType.RiskReward,
#     trail_sl_bool=True,
#     z_or_e_type=None,
# )

# dos_arrays = DynamicOrderSettingsArrays(
#     max_equity_risk_pct=np.array([12]),
#     max_trades=np.array([0]),
#     risk_account_pct_size=np.array([3]),
#     risk_reward=np.array([2, 5]),
#     # sl_based_on_add_pct=np.array([0.1, 0.25, 0.5]),
#     sl_based_on_add_pct=np.array([-0.1, -0.25, -0.5]),  # Use negative percentages for stop-loss
#     sl_based_on_lookback=np.array([20, 50]),
#     sl_bcb_type=np.array([CandleBodyType.High]), # Set stop-loss based on high prices
#     sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
#     sl_to_be_when_pct=np.array([0]),
#     trail_sl_bcb_type=np.array([CandleBodyType.High]), # Use high prices for trailing stop-loss
#     trail_sl_by_pct=np.array([1.0]),
#     trail_sl_when_pct=np.array([1, 2]),
# )

# test
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([5]),
    max_trades=np.array([5]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2,5]),
    sl_based_on_add_pct=np.array([-0.1, -0.25, -0.5]),
    sl_based_on_lookback=np.array([20, 50]),
    sl_bcb_type=np.array([CandleBodyType.High]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.High]),
    trail_sl_by_pct=np.array([1]),
    trail_sl_when_pct=np.array([1, 2]),
)



# dos_arrays = DynamicOrderSettingsArrays(
#     max_equity_risk_pct=np.array([5]),  # (Unchanged)
#     max_trades=np.array([5]),  # (Unchanged)
#     risk_account_pct_size=np.array([3]),  # (Unchanged)
#     risk_reward=np.array([0]),  # (Unchanged)
#     # Key adjustments for short positions:
#     sl_based_on_add_pct=np.array([0, -0.05, -0.1, -0.15]),  # Use negative percentages for stop-loss
#     # sl_based_on_add_pct=np.array([0.1, 0.25, 0.5]),
#     sl_based_on_lookback=np.array([20, 50]),  # (Unchanged)
#     sl_bcb_type=np.array([CandleBodyType.High]),  # Set stop-loss based on high prices
#     sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),  # (Unchanged)
#     sl_to_be_when_pct=np.array([0]),  # (Unchanged)
#     trail_sl_bcb_type=np.array([CandleBodyType.High]),  # Use high prices for trailing stop-loss
#     trail_sl_by_pct=np.array([0.5]),  # (Unchanged)
#     trail_sl_when_pct=np.array([1]),  # (Unchanged)
# )




In [55]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=short_strat,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 24
Total combinations of settings to test: 144
Total candles: 3,000
Total candles to test: 432,000


In [56]:
backtest_results.sort_values(by=["qf_score","total_pnl"], ascending=False).head(10)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,ending_eq,total_pnl
126,5,6,10.0,10,0,34.094,100.0,0.999,7.725,1340.941,340.941
127,5,7,10.0,10,0,34.094,100.0,0.999,7.725,1340.941,340.941
138,5,18,10.0,10,0,34.094,100.0,0.999,7.725,1340.941,340.941
139,5,19,10.0,10,0,34.094,100.0,0.999,7.725,1340.941,340.941
122,5,2,10.0,10,0,33.549,100.0,0.999,7.852,1335.491,335.491
123,5,3,10.0,10,0,33.549,100.0,0.999,7.852,1335.491,335.491
134,5,14,10.0,10,0,33.549,100.0,0.999,7.852,1335.491,335.491
135,5,15,10.0,10,0,33.549,100.0,0.999,7.852,1335.491,335.491
120,5,0,9.0,9,0,29.770,100.0,0.999,10.065,1297.705,297.705
121,5,1,9.0,9,0,29.770,100.0,0.999,10.065,1297.705,297.705


In [57]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=short_strat,
    dos_index=6,
    ind_set_index=5,
    plot_results=True,
    logger_bool=True,
)

DynamicOrderSettings(
    max_equity_risk_pct = 0.05,
    max_trades = 5,
    risk_account_pct_size = 0.03,
    risk_reward = 2.0,
    sl_based_on_add_pct = -0.0025,
    sl_based_on_lookback = 50,
    sl_bcb_type = 2,
    sl_to_be_cb_type = 6,
    sl_to_be_when_pct = 0.0,
    trail_sl_bcb_type = 2,
    trail_sl_by_pct = 0.01,
    trail_sl_when_pct = 0.01,
)
IndicatorSettingsArrays(
    rsi_is_above = 75.0,
    rsi_is_below = nan,
    rsi_length = 14,
)


d:\Lab\quantFreedom\qfstrat\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [10]:
order_records_df


,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,5,31,52,1698742800000,2023-10-31 09:00:00,EntryFilled,1000.000,958.989,2159.203,41.011,...,2200.214,34414.3,NaN,0.064,2200.214,NaN,1.200,33986.1,6.9,36792.2
1,5,31,58,1698764400000,2023-10-31 15:00:00,EntryFilled,1000.000,871.196,7126.951,128.804,...,2896.338,34331.1,NaN,0.148,5096.552,NaN,1.100,34003.4,6.0,36439.6
2,5,31,60,1698771600000,2023-10-31 17:00:00,EntryFilled,1000.000,746.883,13932.106,253.117,...,1961.720,34445.0,NaN,0.205,7058.272,NaN,1.100,34003.4,6.5,36621.9
3,5,31,62,1698778800000,2023-10-31 19:00:00,EntryFilled,1000.000,582.889,22577.894,417.111,...,2004.627,34491.4,NaN,0.263,9062.899,NaN,1.200,34003.4,6.7,36724.3
4,5,31,88,1698872400000,2023-11-01 21:00:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.011,34780.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,5,31,2894,1708974000000,2024-02-26 19:00:00,TakeProfitFilled,4814.026,4814.026,NaN,NaN,...,NaN,NaN,53689.5,NaN,NaN,1798.426,NaN,NaN,NaN,NaN
434,5,31,2967,1709236800000,2024-02-29 20:00:00,EntryFilled,4814.026,4661.279,1570.150,152.747,...,1722.897,61652.6,NaN,0.028,1722.897,NaN,8.200,56570.6,41.9,87480.5
435,5,31,2972,1709254800000,2024-03-01 01:00:00,EntryFilled,4814.026,4354.123,4604.746,459.903,...,1771.602,61600.1,NaN,0.057,3494.499,NaN,8.100,56618.0,41.3,87096.3
436,5,31,2975,1709265600000,2024-03-01 04:00:00,EntryFilled,4814.026,3890.060,9134.091,923.966,...,1958.812,61186.2,NaN,0.089,5453.311,NaN,7.900,56674.1,39.7,85944.1
